In [1]:
import numpy as np
import os
import sys


In [2]:
from easydict import EasyDict as edict
from collections import Counter


In [3]:
from sklearn.metrics import confusion_matrix as sklearn_cm


In [4]:
sys.path.insert(0, '/cluster/tufts/hugheslab/zhuang12/general_utilities')
from shared_utilities import load_pickle, load_txt, save_pickle

In [5]:
def calculate_balanced_accuracy(true_labels, predictions, return_type = 'balanced_accuracy'):
    '''
    used particularly for this 3-classes classification task
    '''
    
    confusion_matrix = sklearn_cm(true_labels, predictions)
    
    class0_recall = confusion_matrix[0,0]/np.sum(confusion_matrix[0])
    class1_recall = confusion_matrix[1,1]/np.sum(confusion_matrix[1])
    class2_recall = confusion_matrix[2,2]/np.sum(confusion_matrix[2])
    
    balanced_accuracy = (1/3)*class0_recall + (1/3)*class1_recall + (1/3)*class2_recall
    
    if return_type == 'all':
        return balanced_accuracy * 100, class0_recall * 100, class1_recall * 100, class2_recall * 100
    elif return_type == 'balanced_accuracy':
        return balanced_accuracy * 100
    else:
        raise NameError('Unsupported return_type in hz_utils calculate_balanced_accuracy fn')


In [6]:
def find_most_frequent(input_array):
    
    occurance_count = Counter(input_array)
    print('input_array:{}, occurance_count.most_common(2):{}'.format(input_array, occurance_count.most_common(2)))
    return occurance_count.most_common(2)[0][0]

In [7]:
def generate_PatientOrder_DataIndicesRange_ImageCount(patients_split_stats_dir, split='val'):
    
    #load the patient_order_list and patient_level_count_dicts pre-generated from Echo_multitask/generate_data/
    #Processing_Echo_method1MaintainingAspectRatio_ResizingThenPad_ExcludeDoppler_grayscale_diagnosis.py
    #Processing_Echo_method1MaintainingAspectRatio_ResizingThenPad_ExcludeDoppler_grayscale_UsedForFullUnlabeledOnly_diagnosis.py
    
    patients_order_list = load_pickle(patients_split_stats_dir, '{}_patient_order_list.pkl'.format(split))
    patientlevel_count_dicts = load_pickle(patients_split_stats_dir, '{}_patient_level_count_dicts.pkl'.format(split))
    
    num_patients = len(patients_order_list)
    patients_ImageCount_list = []
    patients_DataIndicesRange_list = []
    
    for patient_id in patients_order_list:
        this_patient_number_images = 0
        for view, view_labels in patientlevel_count_dicts[patient_id]['view_labels_count'].items():
            this_patient_number_images += view_labels
        
        patients_ImageCount_list.append(this_patient_number_images)
    
    
    patient_DataIndicesEndpoints_list = np.insert(np.cumsum(patients_ImageCount_list), 0, 0)
    
    for i in range(num_patients):
        patients_DataIndicesRange_list.append((patient_DataIndicesEndpoints_list[i], patient_DataIndicesEndpoints_list[i+1]))
        
    
    return patients_order_list, patients_DataIndicesRange_list, patients_ImageCount_list
    
    
    

In [8]:

#this script only compared the selected approach for MLHC paper: SoftMajority vote, Soft probabilistic majority vote
def perform_PatientLevel_integration(args_dict):
    
    fold_idx = args_dict.fold_idx
    split = args_dict.split
    View_predictions_path = args_dict.View_predictions_path
    Diagnosis_predictions_path = args_dict.Diagnosis_predictions_path
    Diagnosis_true_labels_path = args_dict.Diagnosis_true_labels_path
    
    View_predictions = load_pickle(View_predictions_path, 'view_predictions.pkl')
    Diagnosis_predictions = load_pickle(Diagnosis_predictions_path, 'diagnosis_predictions.pkl')
    Diagnosis_true_labels = load_pickle(Diagnosis_true_labels_path, '{}_diagnosis_labels.pkl'.format(split))
    
    #as sanity check:
    if fold_idx == 0:
        set_size = 5617
    elif fold_idx == 1:
        set_size = 5470
    elif fold_idx == 2:
        set_size = 5559
    elif fold_idx == 3:
        set_size = 5628
    else:
        raise NameError('invalid fold_name')
    
    
    total_images = Diagnosis_predictions.shape[0]
    assert total_images == set_size
    
    relevance_threshold = args_dict.relevance_threshold
     
    #get test_patients_order_list, test_patients_DataIndicesRange_list, test_patients_ImageCount_list
    fold_multitask_patients_split_stats_dir = args_dict.fold_multitask_patients_split_stats_dir

    fold_multitask_patients_order_list, fold_multitask_patients_DataIndicesRange_list, fold_multitask_patients_ImageCount_list = generate_PatientOrder_DataIndicesRange_ImageCount(fold_multitask_patients_split_stats_dir, split)


    
    RelevanceFiltered_HardMajorityVote_predicted_labels = []
    patient_true_diagnosis_labels = []
    
    #loop through each patient
    print('Current relevance_threshold: {}'.format(relevance_threshold).center(100,'#'))
    for idx, patient_id in enumerate(fold_multitask_patients_order_list):
        
        print('Currently aggregating predictions for {}'.format(patient_id).center(100, '-'))
        this_patient_data_indices = list(range(total_images))[fold_multitask_patients_DataIndicesRange_list[idx][0]:fold_multitask_patients_DataIndicesRange_list[idx][1]] 
        this_patient_diagnosis_true_labels = Diagnosis_true_labels[this_patient_data_indices]
        
        assert len(list(set(this_patient_diagnosis_true_labels))) == 1, '1 patient can only have 1 diagnosis label'
        this_patient_diagnosis_single_label = this_patient_diagnosis_true_labels[0]

        #record this patient's true diagnosis label
        patient_true_diagnosis_labels.append(this_patient_diagnosis_single_label)
        
        this_patient_diagnosis_predictions = Diagnosis_predictions[this_patient_data_indices]
        this_patient_view_predictions = View_predictions[this_patient_data_indices]
        
        #Suggested Ablation
        this_patient_TargetView_mask = this_patient_view_predictions.argmax(1) != 2 
        this_patient_ConfidenceThreshold_mask = np.sum(this_patient_view_predictions[:,:2], axis=1) > relevance_threshold
        this_patient_TargetViewRelevanceMask = np.logical_and(this_patient_TargetView_mask, this_patient_ConfidenceThreshold_mask)        
        this_patient_TargetView_DiagnosisPredictions = this_patient_diagnosis_predictions[this_patient_TargetViewRelevanceMask]

        if len(this_patient_TargetView_DiagnosisPredictions) > 0:
            this_patient_RelevanceFiltered_HardMajorityVote_predicted_label = find_most_frequent(this_patient_TargetView_DiagnosisPredictions.argmax(1))
        else: #fall back to majority vote
            print('{} at relevance threshold {} fall back to majority vote'.format(patient_id, relevance_threshold))
            this_patient_SoftMajorityVote_prediction = np.mean(this_patient_diagnosis_predictions, axis=0)
            this_patient_RelevanceFiltered_HardMajorityVote_predicted_label = np.argmax(this_patient_SoftMajorityVote_prediction)
            
        RelevanceFiltered_HardMajorityVote_predicted_labels.append(this_patient_RelevanceFiltered_HardMajorityVote_predicted_label)
        print('true_diagnosis: {}, RelevanceFiltered_HardMajorityVote predicted_diagnosis: {}\n'.format(this_patient_diagnosis_single_label, this_patient_RelevanceFiltered_HardMajorityVote_predicted_label))
        
      
    print('\n\n')  
    RelevanceFiltered_HardMajorityVote_balanced_accuracy = calculate_balanced_accuracy(patient_true_diagnosis_labels, RelevanceFiltered_HardMajorityVote_predicted_labels)
    returned_dict = edict()
    returned_dict.true_diagnosis_labels = patient_true_diagnosis_labels
    returned_dict.RelevanceFiltered_HardMajorityVote = {'balanced_accuracy':RelevanceFiltered_HardMajorityVote_balanced_accuracy, 'predicted_labels':np.array(RelevanceFiltered_HardMajorityVote_predicted_labels)}
   
    return returned_dict


### Using fold0 as demo

## FS

In [10]:
fold0_args_dict = edict()
fold0_args_dict.fold_idx = 0
fold0_args_dict.split='val'
fold0_args_dict.View_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/FS'
fold0_args_dict.Diagnosis_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/FS'
fold0_args_dict.Diagnosis_true_labels_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'
fold0_args_dict.fold_multitask_patients_split_stats_dir= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'

candidate_relevance_threshold = np.linspace(0.6, 0.9, 11)
performances = []
for relevance_threshold in candidate_relevance_threshold:
    fold0_args_dict.relevance_threshold = relevance_threshold
    returned_dict = perform_PatientLevel_integration(fold0_args_dict)
    
    performances.append((relevance_threshold, returned_dict.RelevanceFiltered_HardMajorityVote['balanced_accuracy']))
    


##################################Current relevance_threshold: 0.6##################################
---------------------------Currently aggregating predictions for 1183208----------------------------
input_array:[1 2 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0], occurance_count.most_common(2):[(1, 17), (0, 2)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

---------------------------Currently aggregating predictions for 2582424----------------------------
input_array:[1 1 1 1 0 1 1 1 2 2 2 2 2 2 2 2 2 2 2 0 1], occurance_count.most_common(2):[(2, 11), (1, 8)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2875004----------------------------
input_array:[1 1 1 1 1 1 1 1 2 1 0 1 1 0 1 1 1 1 1], occurance_count.most_common(2):[(1, 16), (0, 2)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

----------------------------Currently aggregating 

input_array:[2 2 2 2 2 1 1 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 23), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 1142913----------------------------
input_array:[2 2 2 2 2 1 2 1 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 12), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 1970769----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2], occurance_count.most_common(2):[(2, 15), (1, 1)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2520169----------------------------
input_array:[2 0 2 0 1 1 2 2 2 1 1 2 2], occurance_count.most_common(2):[(2, 7), (1, 4)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagno

input_array:[0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0], occurance_count.most_common(2):[(0, 13), (1, 3)]
true_diagnosis: 0, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 0

---------------------------Currently aggregating predictions for 2583506----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 20), (0, 1)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2498445----------------------------
input_array:[2 2 2], occurance_count.most_common(2):[(2, 3)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 1318207----------------------------
input_array:[2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 19), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

----------------

In [11]:
performances

[(0.6, 86.66666666666666),
 (0.63, 86.66666666666666),
 (0.66, 86.66666666666666),
 (0.69, 88.75),
 (0.72, 88.75),
 (0.75, 86.66666666666666),
 (0.78, 86.66666666666666),
 (0.81, 84.99999999999999),
 (0.8400000000000001, 87.08333333333333),
 (0.8700000000000001, 84.99999999999999),
 (0.9, 88.75)]

## MM

In [11]:
fold0_args_dict = edict()
fold0_args_dict.fold_idx = 0
fold0_args_dict.split='val'
fold0_args_dict.View_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/MixMatch'
fold0_args_dict.Diagnosis_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/MixMatch'
fold0_args_dict.Diagnosis_true_labels_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'
fold0_args_dict.fold_multitask_patients_split_stats_dir= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'

candidate_relevance_threshold = np.linspace(0.6, 0.9, 11)
performances = []
for relevance_threshold in candidate_relevance_threshold:
    fold0_args_dict.relevance_threshold = relevance_threshold
    returned_dict = perform_PatientLevel_integration(fold0_args_dict)
    
    performances.append((relevance_threshold, returned_dict.RelevanceFiltered_HardMajorityVote['balanced_accuracy']))
    


##################################Current relevance_threshold: 0.6##################################
---------------------------Currently aggregating predictions for 1183208----------------------------
input_array:[1 2 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 0 0 0], occurance_count.most_common(2):[(1, 15), (2, 4)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

---------------------------Currently aggregating predictions for 2582424----------------------------
input_array:[1 1 1 1 2 1 1 2 2 2 2 2 2 2 2 2 2 2 0 2], occurance_count.most_common(2):[(2, 13), (1, 6)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2875004----------------------------
input_array:[0 1 1 1 0 2 0 1 2 2 2 1 2 1 2 1 1 2 2], occurance_count.most_common(2):[(1, 8), (2, 8)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

----------------------------Currently aggregating

input_array:[1 2 1 1 1 2 2 1 2 2 2 2], occurance_count.most_common(2):[(2, 7), (1, 5)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2102166----------------------------
input_array:[2 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 17), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2156543----------------------------
input_array:[2 2 2 2 2 1 2 2 1 1 2 2], occurance_count.most_common(2):[(2, 9), (1, 3)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2525420----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 1 1 2 2], occurance_count.most_common(2):[(2, 12), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

-----------------

input_array:[0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0], occurance_count.most_common(2):[(0, 11), (1, 5)]
true_diagnosis: 0, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 0

---------------------------Currently aggregating predictions for 2583506----------------------------
2583506 at relevance threshold 0.81 fall back to majority vote
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2498445----------------------------
input_array:[2 2 1 0 2 2 2 2], occurance_count.most_common(2):[(2, 6), (1, 1)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 1318207----------------------------
input_array:[2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 23), (1, 1)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currentl

In [19]:
performances

[(0.6, 92.5),
 (0.63, 94.58333333333333),
 (0.66, 94.58333333333333),
 (0.69, 94.58333333333333),
 (0.72, 94.58333333333333),
 (0.75, 92.5),
 (0.78, 90.41666666666666),
 (0.81, 92.5),
 (0.8400000000000001, 92.5),
 (0.8700000000000001, 90.83333333333333),
 (0.9, 86.66666666666666)]

## PretrainedMixMatch

In [13]:
fold0_args_dict = edict()
fold0_args_dict.fold_idx = 0
fold0_args_dict.split='val'
fold0_args_dict.View_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/PretrainedMixMatch'
fold0_args_dict.Diagnosis_predictions_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/predictions/ImageLevel_predictions/fold0/val/PretrainedMixMatch'
fold0_args_dict.Diagnosis_true_labels_path= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'
fold0_args_dict.fold_multitask_patients_split_stats_dir= '/cluster/tufts/hugheslab/zhuang12/MLHCCode_Release/split_info/E4VD-156-52/fold0/val'

candidate_relevance_threshold = np.linspace(0.6, 0.9, 11)
performances = []
for relevance_threshold in candidate_relevance_threshold:
    fold0_args_dict.relevance_threshold = relevance_threshold
    returned_dict = perform_PatientLevel_integration(fold0_args_dict)
    
    performances.append((relevance_threshold, returned_dict.RelevanceFiltered_HardMajorityVote['balanced_accuracy']))
    


##################################Current relevance_threshold: 0.6##################################
---------------------------Currently aggregating predictions for 1183208----------------------------
input_array:[2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 0 0 0], occurance_count.most_common(2):[(1, 17), (0, 3)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

---------------------------Currently aggregating predictions for 2582424----------------------------
input_array:[1 1 1 1 2 1 1 1 2 0 2 2 2 2 2 2 2 2 0 2], occurance_count.most_common(2):[(2, 11), (1, 7)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2875004----------------------------
input_array:[1 0 0 0 0 2 0 1 1 2 1 1 2 2 2 1 2 0 2], occurance_count.most_common(2):[(2, 7), (1, 6)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

----------------------------Currently aggregating

input_array:[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 16)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2498445----------------------------
input_array:[2 1 2 2 0 2 2 0 1 2], occurance_count.most_common(2):[(2, 6), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 1318207----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 0], occurance_count.most_common(2):[(2, 23), (1, 2)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2163209----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 13)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

----------------

input_array:[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], occurance_count.most_common(2):[(1, 16)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

---------------------------Currently aggregating predictions for 1490200----------------------------
input_array:[1 2 2 2 1 1 2 0 2 1 1 2 1 2 1], occurance_count.most_common(2):[(1, 7), (2, 7)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 1

---------------------------Currently aggregating predictions for 2850632----------------------------
input_array:[2 2 1 1 2 2 1 1 2 1 2 2 2 2], occurance_count.most_common(2):[(2, 9), (1, 5)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2927071----------------------------
input_array:[1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 2 1 1 2 1 1 1 2 2 2 2], occurance_count.most_common(2):[(1, 19), (2, 6)]
true_diagnosis: 1, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 

input_array:[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 19)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2156543----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 11)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2525420----------------------------
input_array:[2 2 2 2 2 1 2], occurance_count.most_common(2):[(2, 6), (1, 1)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently aggregating predictions for 2551434----------------------------
input_array:[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], occurance_count.most_common(2):[(2, 20)]
true_diagnosis: 2, RelevanceFiltered_HardMajorityVote predicted_diagnosis: 2

---------------------------Currently agg

In [14]:
performances

[(0.6, 87.91666666666667),
 (0.63, 87.91666666666667),
 (0.66, 87.91666666666667),
 (0.69, 87.91666666666667),
 (0.72, 89.99999999999999),
 (0.75, 92.08333333333333),
 (0.78, 92.08333333333333),
 (0.81, 92.08333333333333),
 (0.8400000000000001, 88.33333333333333),
 (0.8700000000000001, 86.25),
 (0.9, 87.91666666666667)]